In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.6/914.6 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from collections import defaultdict
from ultralytics.utils.plotting import Annotator
from PIL import Image

track_history = defaultdict(lambda: [])

model = YOLO("yolov8n.pt")
names = model.model.names
video_path = "/content/vtest.avi.mp4"
output_gif_path = "/content/result.gif"  # Path to save the output GIF

if not Path(video_path).exists():
    raise FileNotFoundError(f"Source path "
                            f"'{video_path}' "
                            f"does not exist.")

cap = cv2.VideoCapture(video_path)

# Get video details for frame processing
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Prepare for GIF saving (collect frames)
frames = []

while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True)

        boxes = results[0].boxes.xywh.cpu()
        clss = results[0].boxes.cls.cpu().tolist()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        annotator = Annotator(frame, line_width=2,
                              example=str(names))

        for box, track_id, cls in zip(boxes, track_ids, clss):
            x, y, w, h = box
            x1, y1, x2, y2 = (x - w / 2, y - h / 2,
                              x + w / 2, y + h / 2)
            label = str(names[cls]) + " : " + str(track_id)
            annotator.box_label([x1, y1, x2, y2],
                                label, (218, 100, 255))

            # Tracking Lines plot
            track = track_history[track_id]
            track.append((float(box[0]), float(box[1])))
            if len(track) > 30:
                track.pop(0)

            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame, [points], isClosed=False,
                          color=(37, 255, 225), thickness=2)

            # Center circle
            cv2.circle(frame,
                       (int(track[-1][0]), int(track[-1][1])),
                       5, (235, 219, 11), -1)

        # Convert the frame from BGR to RGB (PIL uses RGB)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(frame_rgb)

        # Append the frame to the list
        frames.append(pil_image)

        # Optionally, show the frame
        # cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Save frames as GIF
if frames:
    frames[0].save(output_gif_path, save_all=True, append_images=frames[1:], duration=1000 / fps, loop=0)

# Release resources
cap.release()
cv2.destroyAllWindows()


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 122MB/s]


requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.6 MB/s eta 0:00:00

requirements: AutoUpdate success ✅ 3.0s, installed 1 package: ['lap>=0.5.12']
requirements: ⚠️ Restart runtime or rerun command for updates to take effect


0: 480x640 3 persons, 1 car, 1 truck, 336.2ms
Speed: 11.0ms preprocess, 336.2ms inference, 27.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 179.6ms
Speed: 2.9ms preprocess, 179.6ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 184.7ms
Speed: 3.9ms preprocess, 184.7ms inference, 1.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 208.4ms
Speed: 3.1ms preprocess, 208.4ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 182.6ms
Speed: 2.9ms preprocess, 182.6ms inference, 

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from collections import defaultdict
from ultralytics.utils.plotting import Annotator

track_history = defaultdict(lambda: [])

model = YOLO("yolov8n.pt")
names = model.model.names
video_path = "/content/vtest.avi.mp4"
output_video_path = "/content/result.mp4"  # Path to save the output video

if not Path(video_path).exists():
    raise FileNotFoundError(f"Source path "
                            f"'{video_path}' "
                            f"does not exist.")

cap = cv2.VideoCapture(video_path)

# Get video details for output file (frame width, height, fps)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Initialize the VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file format
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True)

        boxes = results[0].boxes.xywh.cpu()
        clss = results[0].boxes.cls.cpu().tolist()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        annotator = Annotator(frame, line_width=2,
                              example=str(names))

        for box, track_id, cls in zip(boxes, track_ids, clss):
            x, y, w, h = box
            x1, y1, x2, y2 = (x - w / 2, y - h / 2,
                              x + w / 2, y + h / 2)
            label = str(names[cls]) + " : " + str(track_id)
            annotator.box_label([x1, y1, x2, y2],
                                label, (218, 100, 255))

            # Tracking Lines plot
            track = track_history[track_id]
            track.append((float(box[0]), float(box[1])))
            if len(track) > 30:
                track.pop(0)

            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame, [points], isClosed=False,
                          color=(37, 255, 225), thickness=2)

            # Center circle
            cv2.circle(frame,
                       (int(track[-1][0]), int(track[-1][1])),
                       5, (235, 219, 11), -1)

        # Write the processed frame to the output video
        out.write(frame)

        # cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()



0: 480x640 3 persons, 1 car, 1 truck, 168.9ms
Speed: 3.9ms preprocess, 168.9ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 170.5ms
Speed: 5.4ms preprocess, 170.5ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 184.6ms
Speed: 6.2ms preprocess, 184.6ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 165.9ms
Speed: 5.3ms preprocess, 165.9ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 158.0ms
Speed: 5.0ms preprocess, 158.0ms inference, 1.1ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 163.0ms
Speed: 5.3ms preprocess, 163.0ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 156.3ms
Speed: 5.3ms preprocess, 156.3ms inference, 1.1ms postprocess per image at sha

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from collections import defaultdict
from ultralytics.utils.plotting import Annotator

track_history = defaultdict(lambda: [])

model = YOLO("yolov8n.pt")
model.to("cpu")
names = model.model.names
video_path = "/content/vtest.avi.mp4"
output_video_path = "/content/bytetrack.mp4"  # Path to save the output video

if not Path(video_path).exists():
    raise FileNotFoundError(f"Source path "
                            f"'{video_path}' "
                            f"does not exist.")

cap = cv2.VideoCapture(video_path)

# Get video details for output file (frame width, height, fps)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Initialize the VideoWriter object to save the video
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for .mp4 file format
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

while cap.isOpened():
    success, frame = cap.read()

    if success:
        results = model.track(frame, persist=True,
                              tracker="bytetrack.yaml")

        boxes = results[0].boxes.xywh.cpu()
        clss = results[0].boxes.cls.cpu().tolist()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        annotator = Annotator(frame, line_width=2,
                              example=str(names))

        for box, track_id, cls in zip(boxes, track_ids, clss):
            x, y, w, h = box
            x1, y1, x2, y2 = (x - w / 2, y - h / 2,
                              x + w / 2, y + h / 2)
            label = str(names[cls]) + " : " + str(track_id)
            annotator.box_label([x1, y1, x2, y2],
                                label, (218, 100, 255))

            # Tracking Lines plot
            track = track_history[track_id]
            track.append((float(box[0]), float(box[1])))
            if len(track) > 30:
                track.pop(0)

            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame, [points], isClosed=False,
                          color=(37, 255, 225), thickness=2)

            # Center circle
            cv2.circle(frame,
                       (int(track[-1][0]), int(track[-1][1])),
                       5, (235, 219, 11), -1)

        # Write the processed frame to the output video
        out.write(frame)

        # cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'yolov10.pt'

In [ ]:
import cv2
import numpy as np
from pathlib import Path
from ultralytics import YOLO
from collections import defaultdict
from ultralytics.utils.plotting import Annotator
from PIL import Image  # Import PIL for GIF saving

track_history = defaultdict(lambda: [])

# Load the YOLOv8 model and set to CPU
model = YOLO("yolov8n.pt")
model.to("cpu")
names = model.model.names
video_path = "/content/vtest.avi.mp4"
output_gif_path = "/content/bytetrack.gif"  # Path to save the output GIF

if not Path(video_path).exists():
    raise FileNotFoundError(f"Source path '{video_path}' does not exist.")

cap = cv2.VideoCapture(video_path)

# Get video details for processing
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# List to store frames for GIF
frames = []

while cap.isOpened():
    success, frame = cap.read()

    if success:
        # Perform tracking using YOLOv8 and ByteTrack
        results = model.track(frame, persist=True, tracker="bytetrack.yaml")

        boxes = results[0].boxes.xywh.cpu()
        clss = results[0].boxes.cls.cpu().tolist()
        track_ids = results[0].boxes.id.int().cpu().tolist()

        annotator = Annotator(frame, line_width=2, example=str(names))

        for box, track_id, cls in zip(boxes, track_ids, clss):
            x, y, w, h = box
            x1, y1, x2, y2 = (x - w / 2, y - h / 2, x + w / 2, y + h / 2)
            label = f"{names[cls]} : {track_id}"
            annotator.box_label([x1, y1, x2, y2], label, (218, 100, 255))

            # Tracking Lines plot
            track = track_history[track_id]
            track.append((float(box[0]), float(box[1])))
            if len(track) > 30:
                track.pop(0)

            points = np.hstack(track).astype(np.int32).reshape((-1, 1, 2))
            cv2.polylines(frame, [points], isClosed=False, color=(37, 255, 225), thickness=2)

            # Draw center circle
            cv2.circle(frame, (int(track[-1][0]), int(track[-1][1])), 5, (235, 219, 11), -1)

        # Convert frame to RGB (PIL format)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        pil_image = Image.fromarray(frame_rgb)

        # Append the frame to the list
        frames.append(pil_image)

        # Optionally, display the frame
        # cv2.imshow("YOLOv8 Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord("q"):
            break
    else:
        break

# Save frames as a GIF
if frames:
    frames[0].save(output_gif_path, save_all=True, append_images=frames[1:], duration=1000 / fps, loop=0)

# Release resources
cap.release()
cv2.destroyAllWindows()



0: 480x640 3 persons, 1 car, 1 truck, 351.2ms
Speed: 18.1ms preprocess, 351.2ms inference, 4.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 1 truck, 344.9ms
Speed: 3.3ms preprocess, 344.9ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 316.2ms
Speed: 3.3ms preprocess, 316.2ms inference, 1.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 211.2ms
Speed: 3.3ms preprocess, 211.2ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 197.7ms
Speed: 3.0ms preprocess, 197.7ms inference, 1.2ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 183.8ms
Speed: 3.8ms preprocess, 183.8ms inference, 1.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 3 persons, 1 car, 2 trucks, 175.4ms
Speed: 3.9ms preprocess, 175.4ms inference, 1.2ms postprocess per image at sh